In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

True

prompt

In [3]:
clarify_with_user_instructions="""
These are the messages that have been exchanged so far from the user asking for the report:
<Messages>
{messages}
</Messages>

Today's date is {date}.

Assess whether you need to ask a clarifying question, or if the user has already provided enough information for you to start research.
IMPORTANT: If you can see in the messages history that you have already asked a clarifying question, you almost always do not need to ask another one. Only ask another question if ABSOLUTELY NECESSARY.

If there are acronyms, abbreviations, or unknown terms, ask the user to clarify.
If you need to ask a question, follow these guidelines:
- Be concise while gathering all necessary information
- Make sure to gather all the information needed to carry out the research task in a concise, well-structured manner.
- Use bullet points or numbered lists if appropriate for clarity. Make sure that this uses markdown formatting and will be rendered correctly if the string output is passed to a markdown renderer.
- Don't ask for unnecessary information, or information that the user has already provided. If you can see that the user has already provided the information, do not ask for it again.

Respond in valid JSON format with these exact keys:
"need_clarification": boolean,
"question": "<question to ask the user to clarify the report scope>",
"verification": "<verification message that we will start research>"

If you need to ask a clarifying question, return:
"need_clarification": true,
"question": "<your clarifying question>",
"verification": ""

If you do not need to ask a clarifying question, return:
"need_clarification": false,
"question": "",
"verification": "<acknowledgement message that you will now start research based on the provided information>"

For the verification message when no clarification is needed:
- Acknowledge that you have sufficient information to proceed
- Briefly summarize the key aspects of what you understand from their request
- Confirm that you will now begin the research process
- Keep the message concise and professional
"""

In [29]:
%%writefile ../deep_research_agent_langgraph/src/state_scope.py

"""State Definitions and Pydantic Schemas for Research Scoping.

This defines the state objects and structured schemas used for
the research agent scoping workflow, including researcher state management and output schemas.
"""



import operator
from typing_extensions import Optional, Annotated, List, Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph import MessagesState
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field

# state definitions

class AgentInputState (MessagesState):

    ''' input state for the full agent, contains message only '''
    pass

class AgentState (MessagesState):

    '''
    Main state for the full multi-agent research system.
        
    Extends MessagesState with additional fields for research coordination. '''

    # research brief generated from user conversation history
    research_breif: Optional[str]
        
    # messages exchanged with the supervisor agent for coordination
    supervisor_messages: Annotated[Sequence[BaseMessage], add_messages ]

    # Raw unprocessed research notes collected during the research phase
    raw_notes: Annotated[List[str], operator.add] = []

    # Processed and structured notes ready for report generation
    notes: Annotated[list[str], operator.add] = []

    # Final formatted research report
    final_report: str


# structured output schemas

class ClarifyWithUser (BaseModel):

    '''Schema for clarifying questions to the user.'''

    need_clarification: bool = Field(
        ..., description="Whether a clarifying question is needed."
    )
    question: str = Field(
        "", description="The clarifying question to ask the user, if needed."
    )
    verification: str = Field(
        "", description="Verification message if no clarification is needed."
    )



Overwriting ../deep_research_agent_langgraph/src/state_scope.py


In [30]:

%%writefile ../deep_research_agent_langgraph/src/research_agent_scope.py


''' user clarification and research breif generation'''

from datetime import datetime
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv
import os
load_dotenv()

from  langchain_core.messages import HumanMessage, AIMessage, get_buffer_string
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command

from deep_research_agent_langgraph.src.prompts import clarify_with_user_instructions, transform_messages_into_research_topic_prompt
from deep_research_agent_langgraph.src.state_scope import AgentState, ClarifyWithUser, ResearchQuestion, AgentInputState
from typing_extensions import Literal

google_api_key = os.getenv("GOOGLE_API_KEY")
if google_api_key:
    os.environ["GOOGLE_API_KEY"] = google_api_key
else:
    
    print("Warning: GOOGLE_API_KEY is not set. Continuing without setting the environment variable.")

# utility func
def get_today_str() -> str:
    """Get current date in a human-readable format."""
    
    now = datetime.now()
    return now.strftime("%a %b {}, %Y").format(now.day)

# config
model = init_chat_model ( "gemini-2.5-flash", model_provider="google_genai", temperature=0)

def clarify_with_user (state: AgentState) -> Command[Literal["write_research_brief", "__end__"]]:

    ''' Clarify the research topic with the user if needed, else generate a research brief.'''

    # structured output model

    structured_output_model = model.with_structured_output(ClarifyWithUser)

    # invoke model with clarification instructions

    response = structured_output_model.invoke ([
        
        HumanMessage(content=clarify_with_user_instructions.format(
            
            messages = get_buffer_string(messages = state["messages"]),
            date = get_today_str()
            ) ) ])

    
    if response.need_clarification:

        return Command(
            goto=START,
            update = { "messages": [AIMessage (content=response.question)] }
        )

    else: 
        return Command(
            goto="write_research_brief",
            update = { "messages": [AIMessage (content=response.verification)] }
        )


def write_research_brief(state: AgentState):
    """
    Transform the conversation history into a comprehensive research brief.
    
    """
    
    structured_output_model = model.with_structured_output(ResearchQuestion)
    
    #  research brief from conversation history
    response = structured_output_model.invoke([
        HumanMessage(content=transform_messages_into_research_topic_prompt.format(
            messages=get_buffer_string(state.get("messages", [])),
            date=get_today_str()
        ))
    ])
    
    # pass to supervisor
    
    return {
        "research_brief": response.research_brief,
        "supervisor_messages": [HumanMessage(content=f"{response.research_brief}.")]
    }



Overwriting ../deep_research_agent_langgraph/src/research_agent_scope.py
